# preparation

In [1]:
import dashscope
import os
import requests
import json
# import torch

In [2]:
# get API key
# dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = 'sk-16f2c80d9d3e4d7d9cc2f6b43f22dfbd' # https://bailian.console.aliyun.com/?tab=model#/api-key
# dashscope.api_key ='sk-qzkddkwofxndanqzonvabhwqbpwganpxkuuoghtxgvmzygnt' # https://cloud.siliconflow.cn/me/account/ak

In [ ]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
# from modelscope import AutoModelForCausalLM, AutoTokenizer

ImportError: cannot import name 'AutoModelForCausalLM' from 'modelscope' (e:\Python\python3.12\Lib\site-packages\modelscope\__init__.py)

# Cases

## 2-2 Mobile phone service subscription

In [ ]:
# use deepseek-v3 model
def get_completion(prompt, model="deepseek-v3"):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format="message",
        temperature=0, # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content

### 使用提示词完成任务

In [ ]:
# system prompt
instructions = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# user prompt
input_text = """办个100G的套餐。"""

prompt = f"""
{instructions}
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# use model to get response
response = get_completion(prompt)
print(response)


### JSON format

In [ ]:
output_format = """以 JSON 格式输出"""

prompt=f"""
{instructions}
{output_format}
{input_text}
"""
response = get_completion(prompt)
print(response)

### CoT case

In [ ]:
instructions = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理

"""

output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

# cot=""
cot="请一步一步分析对话"

prompt=f"""
{instructions}
{cot}
{output_format}
{context}
"""

response=get_completion(prompt)
print(response)

### 使用Prompt调优Prompt

In [ ]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
{instruction}
{user_prompt}
"""

response = get_completion(prompt)
print(response)


## 2-3 DeepSeek-r1-7b

https://lingma.aliyun.com
Qwen3-Coder-480B-A35B-Instruct
480B 完整尺寸的模型参数大小
A35B，激活的参数量是35B

https://modelscope.cn/search?search=deepseek-r1
7B => GPU
1.5B => CPU 内存

https://modelscope.cn/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B

**模型下载**

from modelscope import snapshot_download

snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")

deepseek-ai/DeepSeek-R1-Distill-Qwen-7B => Model ID

方法1：vllm进行部署
方法2：python 调用部署好的模型
方法3：ollama

{'response': '<think>\n\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我是一个AI assistant，通过机器学习技术来模拟人类思考和判断的能力。我理解用户的问题会通过这些正式的、诚实且专业的语言形式呈现，我会以友好、理性的态度为您提供详细的回答。'}

封装成API接口，给其他人用
方法1：fastapi
方法2：flask

约束1：模型尺寸，不会太大
约束2：上网环境

感觉对于我们学员来说，私有化部署是个熟悉使用模型的过程，有助于以后实战微调

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B 与 Ollama运行deepseek-r1:1.5b是一样的吗？
是一样的参数

自己在魔搭平台下的大模型怎么跑？只能用ollama吗
方法1：ollama
方法2：vllm 
方法3：python调用大模型

服务器上怎么部署 调用
Step1，通过modelscope下载对应的大模型

**模型下载**

from modelscope import snapshot_download
snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")


In [ ]:
# %pip install -q modelscope
# from modelscope import snapshot_download

# model_dir = snapshot_download(
# 	'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
# 	cache_dir=r"d:\Job_hunt\2025Courses_LLM\models\deepseek"
# )
# print(f"Model downloaded to: {model_dir}")

In [ ]:
'''
# delete model
import shutil
# shutil.rmtree("/root/autodl-tmp/models") 
shutil.rmtree(r"D:\Job_hunt\2025Courses_LLM\models\deepseek\deepseek-ai")
# d:\Job_hunt\2025Courses_LLM\models\deepseek
'''

In [ ]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
# from modelscope import AutoModelForCausalLM, AutoTokenizer

## Ollama

### Ollama REST API

In [ ]:
import requests

def query_ollama(prompt, model="deepseek-r1:1.5b"):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False  # 设置为 True 可以获取流式响应
    }
    
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        raise Exception(f"API 请求失败: {response.text}")

# 使用示例
response = query_ollama("你好，请介绍一下你自己")
print(response)

### Ollama stream

In [ ]:
def query_ollama_stream(prompt, model="deepseek-r1:1.5b",stream=False):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream":stream # 设置为 True 可以获取流式响应
    }

    if stream:
        # 流式相应处理
        with requests.post(url, json=data, stream=True) as response:
            if response.status_code == 200:
                #  逐行打印流式响应内容
                for line in response.iter_lines(decode_unicode=True):
                    if line:
                        # Ollama流式返回每行是一个json字符串
                        try:
                            # import json
                            obj = json.loads(line)
                            print(obj.get("response",""), end="", flush=True)
                        except Exception as e:
                            print(f"解析流式响应出错: {e}")
            else:
                raise Exception(f"API 请求失败: {response.text}")
            
    else:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            return response.json()["response"]
        else:
            raise Exception(f"API 请求失败: {response.text}")
        
# print("\n=== 流式响应 ===")
# query_ollama_stream("帮我写一个二分查找法",stream=True)

### Ollama FastAPI封装

In [ ]:
# %pip install -q fastapi uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import requests


Note: you may need to restart the kernel to use updated packages.


In [15]:

# ===== Step 1: 定义 FastAPI 应用 =====
app = FastAPI()

# 定义请求模型
class ChatRequest(BaseModel):
    prompt: str
    model: str = "deepseek-r1:1.5b"

# 允许跨域请求（根据需要配置）
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/api/chat")
async def chat(request: ChatRequest):
    ollama_url = "http://localhost:11434/api/generate"
    data = {
        "model": request.model,
        "prompt": request.prompt,
        "stream": False
    }
    
    response = requests.post(ollama_url, json=data)
    if response.status_code == 200:
        return {"response": response.json()["response"]}
    else:
        return {"error": "Failed to get response from Ollama"}, 500

print("✅ FastAPI app 已定义")


✅ FastAPI app 已定义


In [16]:
# ===== Step 2: 在后台启动服务器 =====
import threading
import time

def start_server():
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

# Start server in background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# 等待服务器启动
time.sleep(3)
print("✅ FastAPI server 已在后台启动 (http://localhost:8000)")


e:\Python\python3.12\Lib\site-packages\uvicorn\server.py:67: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return asyncio_run(self.serve(sockets=sockets), loop_factory=self.config.get_loop_factory())
Exception in thread Thread-12 (start_server):
Traceback (most recent call last):
  File "e:\Python\python3.12\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Chailee\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "e:\Python\python3.12\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Chailee\AppData\Local\Temp\ipykernel_3028\755582829.py", line 7, in start_server
  File "e:\Python\python3.12\Lib\site-packages\uvicorn\main.py", line 594, in run
    server.run()
  File "e:\Python\python3.12\Lib\site-packages\uvicorn\server.py", line 67, in run
    return asyncio_run(self.serve(sockets=sockets), loop_factory=s

✅ FastAPI server 已在后台启动 (http://localhost:8000)


In [17]:
# ===== Step 3: 客户端调用 =====
import requests
import time

# 多等一会儿确保服务器已就绪
time.sleep(1)

response = requests.post(
    "http://localhost:8000/api/chat",
    json={
        "model": "deepseek-r1:1.5b",
        "prompt": "你好，请介绍一下你自己",
        "stream": False
    }
)
print(response.json())


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D2A8EA6E10>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))